In [1]:
import h5py
import numpy as np
from scipy.special import expit
import time

# Open the .h5 file in read mode
with h5py.File('data-Mini Project 2.h5', 'r') as file:
    # List all the groups in the file
    trX = file['trX'][:]
    tstX = file['tstX'][:]
    trY = file['trY'][:]
    tstY = file['tstY'][:]

In [2]:
permutation = np.random.permutation(trX.shape[0])
shuffled_train_data = trX
shuffled_labels = trY
for old, new in enumerate(permutation):
    shuffled_train_data[new,:,:] = trX[old,:,:]
    shuffled_labels[new,:] = trY[old,:]


In [3]:
print(shuffled_train_data.shape)
print(shuffled_labels.shape)
print(tstX.shape)
print(tstY.shape)

(3000, 150, 3)
(3000, 6)
(600, 150, 3)
(600, 6)


In [27]:
class RNN:

    def Tanh_activation(self,x):
        return np.tanh(x)
    
    def sigmoid_activation(self,x):
        y = expit(x)
        return y
    
    def Tanh_activation_derivative(self,x):
        return (1/2)*(1-x*x)
    
    def sigmoid_activation_derivative(self,x):
        return (x - x*x) 
    
    
    def initilaze(self, hidden_size, learning_rate, batch_size):
        self.IH_weight = np.random.uniform(-0.01, 0.01, size=(hidden_size, 3))
        self.HH_weight = np.random.uniform(-0.01, 0.01, size=(hidden_size, hidden_size))
        self.HO_weight = np.random.uniform(-0.01, 0.01, size=(6, hidden_size))

        self.learning_rate = learning_rate
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        return

    def train_one_data(self,data_row,result_one):  # data_row means 1 data element (150,3)
        
        self.one_row_data = data_row
        self.result_one = result_one
        self.feedback = np.zeros((self.hidden_size, 1)) # feedback.shape = (50,1)

        self.result_son = self.result_one.reshape(1, 6)
        
        error=0.0
        self.one_data_error = 0.0
        
        #new_column = np.full((150, 1), -1) 
        #data_temp1 = np.concatenate((self.one_row_data, new_column), axis=1)
        #data_temp2 = data_temp1.reshape(150, 4)
        
        EPSILON = 1e-10
        
        for k in range(int(150/self.batch_size)):
            self.h_list = []
            self.y_list = []
            OH_gradient = np.zeros((6, self.hidden_size))
            
            for i in range(self.batch_size):
                data_temp3 = self.one_row_data[self.batch_size * k + i]
                
                data = data_temp3.reshape(3, 1)

                v_1 = np.dot(self.IH_weight,data) + np.dot(self.HH_weight, self.feedback)# v_1 = (50,1)
                activated = self.Tanh_activation(v_1) # activated = (50,1)
                self.feedback = activated # self.feedback = (50,1)
                transposed_activated = activated.T # transposed_activated.shape = (1, 50)
                #add = np.array([-1])
                #transposed_activated_51 = np.append(transposed_activated,add) # transposed_activated_51.shape = (1, hidden_size+1)
                #transposed_activated_51.reshape(1, 51)
                v_2 = np.dot(transposed_activated, self.HO_weight.T) # (1,51) * (51,6) = (1, 6)
                y_result = self.sigmoid_activation(v_2)
                OH_gradient += np.dot((y_result - self.result_one).T, transposed_activated)
                
                error = -self.result_one * np.log10(np.clip(y_result.T, EPSILON, 1 - EPSILON)) - (1 - self.result_one) * np.log10(np.clip(1 - y_result.T, EPSILON, 1 - EPSILON))
                self.one_data_error += np.sum((error))
               
                self.h_list.append(activated.T)      
                self.y_list.append(y_result)
                
            if self.batch_size == 10 or 30: #BACKWARD_10
                self.oh_gradiants = []
                self.hh_gradiants = []
                
                #print(len(self.h_list))
                #print(len(self.y_list))
                
                OH_gradient = np.zeros((6, self.hidden_size))
                #for b in reversed(range(self.batch_size)):
                    #print(b)
                    #OH GRADIENT
                    #d_eksi_y = self.result_son - self.y_list[b]
                    #print(d_eksi_y)
                    #print(d_eksi_y.shape)
                    #OH_gradient_temp = np.dot(-((d_eksi_y).T),self.h_list[b])
                    #print(OH_gradient_temp)
                    #OH_gradient += OH_gradient_temp
                    #self.oh_gradiants.insert(0, OH_gradient)
                    #print(self.y_list[i])
                    #print(self.result_son)
                    #print(d_eksi_y.shape)
                #print(OH_gradient)
                #HH GRADIENT
                start_term_temp = np.dot((self.y_list[self.batch_size - 1] - self.result_son),self.HO_weight)
                #print(start_term_temp.shape)
                start_term = start_term_temp * (1-(self.h_list[self.batch_size - 1])**2)
                #print(start_term.shape)
                
                self.h_list = self.h_list[:-1]
                #print(len(self.h_list))
                
                index = 0
                HH_gradient = np.zeros((self.hidden_size ,self.hidden_size))
                IH_gradient = np.zeros((self.hidden_size,3))
                
                for g in reversed(range(1,len(self.h_list))):
                    #print(g)
                    repeat = start_term
                    for j in range(len(self.h_list)-1,len(self.h_list)-index-1,-1):
                        repeat = np.dot(repeat,self.HH_weight) * (1-self.h_list[j]**2) #term = (term @ WHH) * h[j]
                    
                    HH_gradient += np.dot(repeat.T,self.h_list[g])
                    IH_gradient += np.dot(repeat.T,data.T)
                    index += 1
                
                OH_gradient_clipped = np.clip(OH_gradient, -0.5, 0.5)
                HH_gradient_clipped = np.clip(HH_gradient, -0.5, 0.5)
                IH_gradient_clipped = np.clip(IH_gradient, -0.5, 0.5)
                
                #print(OH_gradient_clipped)
                #print(HH_gradient_clipped)
                #print(IH_gradient_clipped)
                
                
                self.HO_weight -= self.learning_rate * OH_gradient_clipped/self.batch_size
                self.HH_weight -= self.learning_rate * HH_gradient_clipped/self.batch_size
                self.IH_weight -= self.learning_rate * IH_gradient_clipped/self.batch_size
                                
        return self.one_data_error
    
    
    def acc_calc_top1(self,y, rs):
        
        max_of_forward = np.argmax(y)
        max_of_label = np.argmax(rs)
        if max_of_forward == max_of_label:
            return 1
        else:
            return 0
        
    def acc_calc_top2(self,y, rs):
        
        max_of_forward = np.argmax(y)
        y[max_of_forward] = -99999
        second_max_of_forward = np.argmax(y)
        max_of_label = np.argmax(rs)
        if max_of_label ==  second_max_of_forward :
            return 1
        else:
            return 0  
        
        
    def acc_calc_top3(self,y, rs):
        
        max_of_forward = np.argmax(y)
        y[max_of_forward] = -99999
        second_max_of_forward = np.argmax(y)
        y[second_max_of_forward] = -99999
        third_max_of_forward = np.argmax(y)
        max_of_label = np.argmax(rs)
        if max_of_label == third_max_of_forward:
            return 1
        else:
            return 0  
    
    
    def TEST_one_data(self,data_row,result_one):  # data_row means 1 data element (150,3)
        
        self.one_row_data = data_row
        self.result_one = result_one
        self.result_son = self.result_one.reshape(1, 6)
        error=0.0
        self.one_data_error = 0.0
        self.feedback = np.zeros((self.hidden_size, 1)) # feedback.shape = (50,1)
        EPSILON = 1e-10
        
        for k in range(150): 
            
            data_temp3 = self.one_row_data[k]
            data = data_temp3.reshape(3, 1)

            v_1 = np.dot(self.IH_weight,data) + np.dot(self.HH_weight, self.feedback)# v_1 = (50,1)
            activated = self.Tanh_activation(v_1) # activated = (50,1)
            self.feedback = activated # self.feedback = (50,1)
            transposed_activated = activated.T # transposed_activated.shape = (1, 50)
            #add = np.array([-1])
            #transposed_activated_51 = np.append(transposed_activated,add) # transposed_activated_51.shape = (1, hidden_size+1)
            #transposed_activated_51.reshape(1, 51)
            v_2 = np.dot(transposed_activated, self.HO_weight.T) # (1,51) * (51,6) = (1, 6)
            y_result = self.sigmoid_activation(v_2) 
            #error += -self.result_one * np.log10(np.clip(y_result.T, EPSILON, 1 - EPSILON)) - (1 - self.result_one) * np.log10(np.clip(1 - y_result.T, EPSILON, 1 - EPSILON))
            #self.one_data_error += np.sum(error)
        
        top_1 = self.acc_calc_top1(y_result.T, self.result_son)
        top_2 = self.acc_calc_top2(y_result.T, self.result_son)
        top_3 = self.acc_calc_top3(y_result.T, self.result_son)
        return top_1, top_2, top_3
    

In [36]:
print("---------------------------------1-------------------------------------")
print("Full Data Training\nHidden Size:50\nLearning Rate:0.05\nMini Batch Size:10")
network = RNN()
network.initilaze(50, 0.05, 10)
data_size = 3000


acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
        
print("\nBEFORE TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100),"\n")


startTime = time.time()
for epoch in range(50):
    print("epoch: ", epoch + 1)
    cum_error=0.0
    for data_index in range(data_size):
        onedata_error = network.train_one_data(shuffled_train_data[data_index], shuffled_labels[data_index]) 
        #print(onedata_error)
        cum_error+= onedata_error
        
    print("Error after epoch,", epoch + 1, "is:",(cum_error/(data_size*150)))
    
    
acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
stopTime = time.time()
print("\n")
print(stopTime - startTime,"Seconds\n")

print("AFTER TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100))
print("\n---------------------------------2-------------------------------------")
print("Full Data Training\nHidden Size:50\nLearning Rate:0.05\nMini Batch Size:30")
network = RNN()
network.initilaze(50, 0.05, 30)
data_size = 3000


acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
        
print("\nBEFORE TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100),"\n")


startTime = time.time()
for epoch in range(50):
    print("epoch: ", epoch + 1)
    cum_error=0.0
    for data_index in range(data_size):
        onedata_error = network.train_one_data(shuffled_train_data[data_index], shuffled_labels[data_index]) 
        #print(onedata_error)
        cum_error+= onedata_error
        
    print("Error after epoch,", epoch + 1, "is:",(cum_error/(data_size*150)))
    
    
acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
stopTime = time.time()
print("\n")
print(stopTime - startTime,"Seconds\n")

print("AFTER TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100))
print("\n--------------------------------3--------------------------------------")
print("Full Data Training\nHidden Size:50\nLearning Rate:0.01\nMini Batch Size:10")
network = RNN()
network.initilaze(50, 0.01, 10)
data_size = 3000


acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
        
print("\nBEFORE TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100),"\n")


startTime = time.time()
for epoch in range(50):
    print("epoch: ", epoch + 1)
    cum_error=0.0
    for data_index in range(data_size):
        onedata_error = network.train_one_data(shuffled_train_data[data_index], shuffled_labels[data_index]) 
        #print(onedata_error)
        cum_error+= onedata_error
        
    print("Error after epoch,", epoch + 1, "is:",(cum_error/(data_size*150)))
    
    
acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
stopTime = time.time()
print("\n")
print(stopTime - startTime,"Seconds\n")

print("AFTER TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100))

print("\n---------------------------------4-------------------------------------")
print("Full Data Training\nHidden Size:50\nLearning Rate:0.01\nMini Batch Size:30")
network = RNN()
network.initilaze(50, 0.01, 30)
data_size = 3000


acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
        
print("\nBEFORE TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100),"\n")


startTime = time.time()
for epoch in range(50):
    print("epoch: ", epoch + 1)
    cum_error=0.0
    for data_index in range(data_size):
        onedata_error = network.train_one_data(shuffled_train_data[data_index], shuffled_labels[data_index]) 
        #print(onedata_error)
        cum_error+= onedata_error
        
    print("Error after epoch,", epoch + 1, "is:",(cum_error/(data_size*150)))
    
    
acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
stopTime = time.time()
print("\n")
print(stopTime - startTime,"Seconds\n")

print("AFTER TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100))

print("\n---------------------------------5-------------------------------------")
print("Full Data Training\nHidden Size:100\nLearning Rate:0.05\nMini Batch Size:10")
network = RNN()
network.initilaze(100, 0.05, 10)
data_size = 3000


acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
        
print("\nBEFORE TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100),"\n")


startTime = time.time()
for epoch in range(50):
    print("epoch: ", epoch + 1)
    cum_error=0.0
    for data_index in range(data_size):
        onedata_error = network.train_one_data(shuffled_train_data[data_index], shuffled_labels[data_index]) 
        #print(onedata_error)
        cum_error+= onedata_error
        
    print("Error after epoch,", epoch + 1, "is:",(cum_error/(data_size*150)))
    
    
acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
stopTime = time.time()
print("\n")
print(stopTime - startTime,"Seconds\n")

print("AFTER TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100))

print("\n---------------------------------6-------------------------------------")
print("Full Data Training\nHidden Size:100\nLearning Rate:0.05\nMini Batch Size:30")
network = RNN()
network.initilaze(100, 0.05, 30)
data_size = 3000


acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
        
print("\nBEFORE TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100),"\n")


startTime = time.time()
for epoch in range(50):
    print("epoch: ", epoch + 1)
    cum_error=0.0
    for data_index in range(data_size):
        onedata_error = network.train_one_data(shuffled_train_data[data_index], shuffled_labels[data_index]) 
        #print(onedata_error)
        cum_error+= onedata_error
        
    print("Error after epoch,", epoch + 1, "is:",(cum_error/(data_size*150)))
    
    
acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
stopTime = time.time()
print("\n")
print(stopTime - startTime,"Seconds\n")

print("AFTER TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100))

print("\n-------------------------------7---------------------------------------")
print("Full Data Training\nHidden Size:100\nLearning Rate:0.01\nMini Batch Size:10")
network = RNN()
network.initilaze(100, 0.01, 10)
data_size = 3000


acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
        
print("\nBEFORE TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100),"\n")


startTime = time.time()
for epoch in range(50):
    print("epoch: ", epoch + 1)
    cum_error=0.0
    for data_index in range(data_size):
        onedata_error = network.train_one_data(shuffled_train_data[data_index], shuffled_labels[data_index]) 
        #print(onedata_error)
        cum_error+= onedata_error
        
    print("Error after epoch,", epoch + 1, "is:",(cum_error/(data_size*150)))
    
    
acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
stopTime = time.time()
print("\n")
print(stopTime - startTime,"Seconds\n")

print("AFTER TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100))

print("\n--------------------------------8--------------------------------------")
print("Full Data Training\nHidden Size:100\nLearning Rate:0.01\nMini Batch Size:30")
network = RNN()
network.initilaze(100, 0.01, 30)
data_size = 3000


acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
        
print("\nBEFORE TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100),"\n")


startTime = time.time()
for epoch in range(50):
    print("epoch: ", epoch + 1)
    cum_error=0.0
    for data_index in range(data_size):
        onedata_error = network.train_one_data(shuffled_train_data[data_index], shuffled_labels[data_index]) 
        #print(onedata_error)
        cum_error+= onedata_error
        
    print("Error after epoch,", epoch + 1, "is:",(cum_error/(data_size*150)))
    
    
acc_sum_1 = 0
acc_sum_2 = 0
acc_sum_3 = 0
for data_index in range(600):
        acc_top_1, acc_top_2, acc_top_3 = network.TEST_one_data(tstX[data_index], tstY[data_index])
        acc_sum_1 += acc_top_1
        acc_sum_2 += acc_top_2
        acc_sum_3 += acc_top_3
stopTime = time.time()
print("\n")
print(stopTime - startTime,"Seconds\n")

print("AFTER TRAINING:")
print("TOP 1 ACCURACY: %",((acc_sum_1/600)*100))
print("TOP 2 ACCURACY: %",(((acc_sum_1 + acc_sum_2)/600)*100))
print("TOP 3 ACCURACY: %",(((acc_sum_1 + acc_sum_2 + acc_sum_3)/600)*100))
print("\n--------------------------------END--------------------------------------")



---------------------------------1-------------------------------------
Full Data Training
Hidden Size:50
Learning Rate:0.05
Mini Batch Size:10

BEFORE TRAINING:
TOP 1 ACCURACY: % 17.0
TOP 2 ACCURACY: % 31.166666666666664
TOP 3 ACCURACY: % 48.0 

epoch:  1
Error after epoch, 1 is: 10.757270721199973
epoch:  2
Error after epoch, 2 is: 10.73214954997252
epoch:  3
Error after epoch, 3 is: 10.724463915513141
epoch:  4
Error after epoch, 4 is: 10.673650546863685
epoch:  5
Error after epoch, 5 is: 10.654991458446622
epoch:  6
Error after epoch, 6 is: 10.656929621248562
epoch:  7
Error after epoch, 7 is: 10.65334229890904
epoch:  8
Error after epoch, 8 is: 10.652235176231539
epoch:  9
Error after epoch, 9 is: 10.64140277734639
epoch:  10
Error after epoch, 10 is: 10.634520021648934
epoch:  11
Error after epoch, 11 is: 10.62971863410628
epoch:  12
Error after epoch, 12 is: 10.619164097910925
epoch:  13
Error after epoch, 13 is: 10.617767801913923
epoch:  14
Error after epoch, 14 is: 10.6158328